In [13]:
# Exam 2020/06/27 - Exercise #2 - SQL-based version #2
# Alternative solution completely based on Spark SQL

In [14]:
inputPathCars = "exam_ex2_data/Cars.txt"
inputPathFailures = "exam_ex2_data/CarsFailures.txt"
outputPathPart1 = "outPart1"
outputPathPart2 = "outPart2"

In [15]:
# *****************************************
# Exercise 2 - Part 1
# *****************************************

In [16]:
# Read the content of CarsFailures.txt
failuresDF = spark.read.load(inputPathFailures,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "date")\
.withColumnRenamed("_c1", "time")\
.withColumnRenamed("_c2", "carId")\
.withColumnRenamed("_c3", "failureType")\
.cache()

In [17]:
# Read the content of Cars.txt
carsDF = spark.read.load(inputPathCars,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "carId")\
.withColumnRenamed("_c1", "model")\
.withColumnRenamed("_c2", "company")\
.withColumnRenamed("_c3", "city")

In [18]:
# Associate the two dataFrames to two temporary tables
carsDF.createOrReplaceTempView("cars")
failuresDF.createOrReplaceTempView("failures")

In [19]:
# Define a UDF that returns 1 if the year is 2018. 0 otherwise.
def year2018Func(date):
    if date.startswith("2018"):
        return 1
    else:
        return 0
    

spark.udf.register("year2018", year2018Func)

<function __main__.year2018Func(date)>

In [20]:
# Define a UDF that returns 1 if the year is 2017. 0 otherwise.
def year2017Func(date):
    if date.startswith("2017"):
        return 1
    else:
        return 0
    

spark.udf.register("year2017", year2017Func)

<function __main__.year2017Func(date)>

In [21]:
# Select only the failures related to years 2017 and 2018 and failureType=Engine
# Count 
# - the number of engine failures in year 2017
# - the number of engine failures in year 2018 
# Select only the car with #failures 2018 > #failures 2017

selectedCarsModelsDF = spark.sql("""SELECT cars.carId, model
FROM cars, failures
WHERE cars.carId=failures.carId
AND failures.failureType='Engine'
GROUP BY cars.carId, model
HAVING SUM( year2018(date)) > SUM( year2017(date)) """)

In [24]:
# Store the result
selectedCarsModelsDF.write.csv(outputPathPart1,header=False)

In [27]:
# *****************************************
# Exercise 2 - Part 2
# *****************************************

In [28]:
# This function is already provided 
from datetime import datetime, timedelta
def previousDate(mydate):
    currentDate=datetime.strptime(mydate,"%Y/%m/%d")
    prevDate=currentDate-timedelta(days=1)
    return prevDate.strftime("%Y/%m/%d")

In [30]:
# Define a UDF that returns the previous date
spark.udf.register("prevDate", lambda currDate: previousDate(currDate))

<function __main__.<lambda>(currDate)>

In [31]:
# Select only carId and Date from each failure
# and remove duplicates
carsDatesDF = failuresDF.select("carId", "Date").distinct()

In [35]:
# Create a new dataframe that contains one record carId, Date-1 for each record of carsDatesDF
# This is used to specify that each record of carsDatesDF is the second element of a window that 
# starts at Date-1
carsDatesYesterdayDF = carsDatesDF.selectExpr("carId", "prevDate(Date) as Date")

In [38]:
# Apply union on the two dataFrames
windowsElementsDF = carsDatesDF.union(carsDatesYesterdayDF)

In [48]:
# Associate windowsElementsDF to temporary table
windowsElementsDF.createOrReplaceTempView("windowElements")

In [49]:
# Select only the groups (carId, Date) associated with two records. These are associated with the 
# windows of two consecutive dates with at least one failure 
selectedWindowsDF = spark.sql("""SELECT carId,Date
FROM windowElements
GROUP BY carId,Date
HAVING COUNT(*)=2""")

In [56]:
# Store the result
selectedWindowsDF.write.csv(outputPathPart2,header=False)